In [7]:
import requests
from bs4 import BeautifulSoup as soup
from bs4 import element
import re
import lxml

## Getting links and metadata from https://www.n-tv.de/news.xml

In [4]:
def extract_meta_data():
    news = soup(requests.get('https://www.n-tv.de/news.xml').text, features="xml")
    
    keywords = ['Ukraine','Russland', 'Wladimir Putin']  # ToDo Keywords spezifizieren
    
    filtered_news = [url for url in news.find('urlset').find_all('url', recursive=False) if len(set(keywords).intersection(set(url.find('news:keywords').text.split(', '))))!=0]
    
    return_list = []
    for new in filtered_news:
        dictionary = {
        'url' : new.find('loc').text,
        'title' : new.find('news:title').text,
        'date' : new.find('news:publication_date').text,
        'keywords' : new.find('news:keywords').text
        }
        dictionary = { **dictionary, 'figcaption' : new.find('image:caption').text } if new.find('image:caption') else dictionary
        return_list += [dictionary]
        
    return return_list


## Getting text and ... from urls

In [131]:
url = extract_meta_data()[5]['url']
url

'https://www.n-tv.de/panorama/Hunderte-ukrainische-Lehrer-bewerben-sich-in-Deutschland-article23286199.html'

In [43]:
type(list(article_soup.find('span', class_='article__kicker').children)[0])
isinstance(list(article_soup.find('span', class_='article__kicker').children)[0],element.NavigableString)

True

In [41]:
list(article_soup.find('span', class_='article__kicker').children)[0].name

In [8]:
def get_article_content(url):
    article_soup = soup(requests.get(url).text)
    kicker = article_soup.find('span', class_='article__kicker').text
    title =  article_soup.find('span', class_='article__headline').text
    author = article_soup.find('span', class_='article__author').text if article_soup.find('span', class_='article__author') else ''
    date = article_soup.find('span', class_='article__date').text
    strong = article_soup.find('strong').text if article_soup.find('strong') else ''
    article_text = article_soup.find('div', class_='article__text')
    text = strong + ' '.join([x for p in article_text.find_all(lambda x: x.name =='p') for x in [i for i in p.contents if isinstance(i,element.NavigableString) if i]])
    subheadline = '. '.join([h2.text for h2 in article_text.find_all('h2')])
    
    return_dict = {
        'title': title,
        'kicker': kicker,
        'author':author,
        'date':date,
        'text':text,
        'subheadlines': subheadline,
        'url': url
    }
    
    return return_dict


In [9]:
get_article_content(extract_meta_data()[5]['url'])

{'title': 'Ukraine: Über 3800 zivile Todesopfer seit Ausbruch des russischen Angriffskrieges',
 'kicker': '\n                    Dunkelziffer wohl deutlich höher',
 'author': '',
 'date': '25.04.2022, 12:22 Uhr',
 'text': 'Durch den vor gut zwei Monaten begonnenen russischen Angriffskrieg in der Ukraine sind nach ukrainischen Angaben mindestens 3818 Zivilisten getötet worden. "Verletzte: mehr als 4000", teilte die ukrainische Generalstaatsanwältin Iryna Wenediktowa der Nachrichtenagentur Interfax-Ukraine am Montag mit. (Foto: dpa)  Die Statistik sei jedoch unvollständig, da die Behörden zu vielen Orten, darunter zur blockierten Hafenstadt Mariupol, keinen Zugang hätten. Aktuell geht die Staatsanwaltschaft dabei von mindestens 215 getöteten und 391 verletzten Kindern aus. Die Vereinten Nationen haben bisher rund 2500 zivile Tote erfasst, gehen aber ebenso wie Kiew von weitaus höheren zivilen Opferzahlen aus. ',
 'subheadlines': '',
 'url': 'https://www.n-tv.de/ticker/Ukraine-Uber-3800-z